# Simon's Algorithm
[Brief description of algorithm]

# References 
[1] references here with hyperlinkns

# Run on a local simulator

In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

from simons_algorithm import simons_oracle, simons_circuit, submit_simons_task, process_simons_results
from braket.aws import AwsDevice
from braket.devices import LocalSimulator
from braket.tracking import Tracker

tracker = Tracker().start()

In [2]:
n = 5
s = "".join(npr.choice(["0","1"], size=n))

oracle = simons_oracle(s)

# Print the circuits

In [3]:
print(simons_circuit(oracle))


T  : |0|    1    | 2 |3|4|
                          
q0 : -H-C---------C---C-H-
        |         |   |   
q1 : -H-|-C-------|-H-|---
        | |       |   |   
q2 : -H-|-|-C-----|-H-|---
        | | |     |   |   
q3 : -H-|-|-|-C---|-H-|---
        | | | |   |   |   
q4 : -H-|-|-|-|-C-|-H-|---
        | | | | | |   |   
q5 : ---X-|-|-|-|-X---|---
          | | | |     |   
q6 : -----X-|-|-|-----X---
            | | |         
q7 : -------X-|-|---------
              | |         
q8 : ---------X-|---------
                |         
q9 : -----------X---------

T  : |0|    1    | 2 |3|4|


In [4]:
local_simulator = LocalSimulator()

task = submit_simons_task(oracle, local_simulator)

In [5]:
secret_revealed = process_simons_results(task)
print("Result string:", secret_revealed)
print("Secret string", s)

Result string: 11000
Secret string 11000


# Run on a QPU

Here we run the Simon's algorithm on OQC Lucy device. The cost to run this experiment is \$0.3 per task and \$0.00035 per shot. When submitting a circuit, we specify a number of shots that is twice the number of qubits (i.e. four times the length of the secret string). For a string of 5 characters, the total cost is \$0.30 USD.

In [6]:
# Code here
oqc_lucy = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")
task = submit_simons_task(oqc_lucy)

In [7]:
secret_revealed = process_simons_results(task)
print("Result string:", secret_revealed)
print("Secret string", s)

AttributeError: 'str' object has no attribute 'keys'

In [8]:
print(
    f"Estimated cost to run this example: {tracker.qpu_tasks_cost() + tracker.simulator_tasks_cost():.2f} USD"
)

Estimated cost to run this example: 0.00 USD


Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).